In [ ]:
%pip install pandas --upgrade

In [ ]:
import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time

# Main

In [ ]:

import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time
import datetime



def init():
    mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 Terminal\terminal64.exe")
    mt5.login(89045253 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Demo')

# return balance , equity , profit
def info():

    account_info=mt5.account_info()

    balance = account_info.balance
    equity  = account_info.equity
    profit  = account_info.profit
    
    return balance , equity , profit
    

def symbol_info(symbol):
    dic_symbol_info = {}
    group_symbols=mt5.symbols_get(group='*USD*')
    for s in group_symbols:
        if s.name == symbol : 
            dic_symbol_info[s.name] =  s.ask , s.bid
            
    return dic_symbol_info  # برمیگرداند یک دیکشنری که کلدی ها نماد و در ولیو ها اسک و بید به صورت تاپل 


def buy(symbol , lot , tp_ , sl_ ):
    
    
    tp = abs(mt5.symbol_info_tick(symbol).bid + tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).ask - sl_ ) 
    pos = mt5.ORDER_TYPE_BUY







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
    print(request)   
    # send a trading request
    result = mt5.order_send(request)
    
    # check the execution result
    print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def sell(symbol , lot , tp_ , sl_ ):
    
    tp = abs(mt5.symbol_info_tick(symbol).ask - tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).bid + sl_ ) 
    pos = mt5.ORDER_TYPE_SELL







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
        
    # send a trading request
    result = mt5.order_send(request)
    # check the execution result
    # print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def trade(symbol , lot , tp , sl):
    numtrade = 0
    # up_numtrade = 0
    sl = sl * 5000
    while True:
        
        positions_total=len(mt5.positions_get(symbol=symbol))
        print(positions_total)
        if  positions_total == 0 :
            
            th.Thread(target= buy , args= ( symbol, lot ,  tp ,  sl )).start()
            th.Thread(target= sell , args= ( symbol, lot ,  tp ,  sl)).start()
            
            numtrade = 2
            continue
        
        if  positions_total != None :
            if positions_total < numtrade :
                th.Thread(target= buy  , args= ( symbol, lot ,  tp ,  sl )).start()
                th.Thread(target= sell , args= ( symbol, lot ,  tp ,  sl)).start()
                
                
                numtrade =+ 2
                continue
            

            
            numtrade = positions_total
        else:
            print("Positions not found")

        time.sleep(0.5)
        
        
def main():
    # دریافت زمان فعلی



    print(info())
    # th.Thread(target= trade , args= ( 'USDJPY_o' ,  0.1 , 0.009   , 0.006  )).start()
    th.Thread(target= trade , args= ( 'GBPUSD_o' ,  0.1 , 0.00009 , 0.005 )).start()
    # th.Thread(target= trade , args= ( 'NZDUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCAD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCHF_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'EURUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()


    # trade(symbol='USDJPY_o' , lot= 0.1 ,tp= 0.009 ,sl= 0.006)
    # trade(symbol='GBPUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='NZDUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCAD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCHF_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='EURUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)


def close():
    
    
    positions = mt5.positions_get()
    def close_position(position):

        tick = mt5.symbol_info_tick(position.symbol)

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
            "price": tick.ask if position.type == 1 else tick.bid,  
            "deviation": 100,
            "magic": 234002,
            "comment": "Gu",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK ,
        }

        result = mt5.order_send(request)
        return result

    for position in positions:
        # print(position)
        time.sleep(0.01)
        close_position(position)


def run():


    while True:
        
          # توقف اجرا برای ۶۰ ثانیه
        current_time = datetime.datetime.now().time()
        print(current_time)
        # مقایسه زمان فعلی با زمان مورد نظر (ساعت 6:00 صبح)
        target_time = datetime.time(6, 1, 0)  # زمان مورد نظر
        target_close = datetime.time(19, 26, 0)  # زمان مورد نظر
        run_main = 0
        close_main = 0

        if current_time.hour == target_time.hour and run_main == 0  :
            run_main = 1

            main()
            
        
        if current_time.hour == target_close.hour and close_main == 0 :
            close_main = 1

            close()
            
            
        time.sleep(60)

init()
main()


In [3]:
import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time
import datetime

pos_list = []
open_pos_list = []



def init():
    mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 Terminal\terminal64.exe")
    mt5.login(89051664 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Demo')

# return balance , equity , profit
def info():

    account_info=mt5.account_info()

    balance = account_info.balance
    equity  = account_info.equity
    profit  = account_info.profit
    
    return balance , equity , profit
    

def symbol_info(symbol):
    dic_symbol_info = {}
    group_symbols=mt5.symbols_get(group='*USD*')
    for s in group_symbols:
        if s.name == symbol : 
            dic_symbol_info[s.name] =  s.ask , s.bid
            
    return dic_symbol_info  # برمیگرداند یک دیکشنری که کلدی ها نماد و در ولیو ها اسک و بید به صورت تاپل 


def buy(symbol , lot , tp_ , sl_ ):
    
    
    tp = abs(mt5.symbol_info_tick(symbol).bid + tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).ask - sl_ ) 
    pos = mt5.ORDER_TYPE_BUY


    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
      
    # send a trading request
    result = mt5.order_send(request)
    pos_list.append(result.order)
    # print(result)
    # check the execution result
    # print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def sell(symbol , lot , tp_ , sl_ ):
    
    tp = abs(mt5.symbol_info_tick(symbol).ask - tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).bid + sl_ ) 
    pos = mt5.ORDER_TYPE_SELL







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
        
    # send a trading request
    result = mt5.order_send(request)
    pos_list.append(result.order)
    # check the execution result
    # print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def dif_pos(symbol):
    
    while True:
        open_pos = mt5.positions_get(symbol=symbol)
        time.sleep(0.1)
        for pos in open_pos:
            if pos.ticket not in open_pos_list:
                open_pos_list.append(pos.ticket)
        return open_pos_list
    

def trade(symbol , lot , tp , sl):
    numtrade = 0
    
    # up_numtrade = 0
    # sl = sl * 5000
    while True:
        
        positions_total = len(mt5.positions_get(symbol=symbol))
        positions_info  = mt5.positions_get(symbol=symbol)
        
        # print(positions_total)
        if  positions_total == 0 :
            
            th.Thread(target= buy , args= ( symbol, lot ,  tp ,  sl )).start()
            th.Thread(target= sell , args= ( symbol, lot ,  tp ,  sl)).start()
            
            numtrade = 2
            continue
        # print(positions_info[0].type)
        
        
        
        
        if  positions_total != None :
            if positions_total < numtrade :
                if positions_info[0].type == 1 : 
                    buy( symbol, lot ,  tp ,  sl*2   )
                    sell( symbol, lot ,  tp ,  sl)
                if positions_info[0].type == 0 : 
                    sell( symbol, lot ,  tp ,  sl* 2)
                    buy( symbol, lot ,  tp ,  sl   )
                
                
                
                
                numtrade =+ 2
                continue
            

            
            numtrade = positions_total
        

        time.sleep(0.1)
        
        
def main():
    # دریافت زمان فعلی



    print(info())
    # th.Thread(target= trade   , args= ( 'USDJPY_o' ,  0.1 , 0.009   , 0.006   )).start()
    th.Thread(target= trade , args= ( 'GBPUSD_o' ,  0.1 , 0.0004 , 0.0002 )).start()
    # th.Thread(target= trade , args= ( 'NZDUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCAD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCHF_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'EURUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()


    # trade(symbol='USDJPY_o' , lot= 0.1 ,tp= 0.009 ,sl= 0.006)
    # trade(symbol='GBPUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='NZDUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCAD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCHF_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='EURUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)


def close():
    
    
    positions = mt5.positions_get()
    def close_position(position):

        tick = mt5.symbol_info_tick(position.symbol)

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
            "price": tick.ask if position.type == 1 else tick.bid,  
            "deviation": 100,
            "magic": 234002,
            "comment": "Gu",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK ,
        }

        result = mt5.order_send(request)
        return result

    for position in positions:
        # print(position)
        time.sleep(0.01)
        close_position(position)


def run():


    while True:
        
          # توقف اجرا برای ۶۰ ثانیه
        current_time = datetime.datetime.now().time()
        print(current_time)
        # مقایسه زمان فعلی با زمان مورد نظر (ساعت 6:00 صبح)
        target_time = datetime.time(6, 1, 0)  # زمان مورد نظر
        target_close = datetime.time(19, 26, 0)  # زمان مورد نظر
        run_main = 0
        close_main = 0

        if current_time.hour == target_time.hour and run_main == 0  :
            run_main = 1

            main()
            
        
        if current_time.hour == target_close.hour and close_main == 0 :
            close_main = 1

            close()
            
            
        time.sleep(60)

init()
main()

(96317.05, 96316.05, -1.0)


Exception in thread Thread-6 (trade):
Traceback (most recent call last):
  File "C:\Users\delta2794900\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\delta2794900\Documents\env\aj\.venv\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\delta2794900\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\delta2794900\AppData\Local\Temp\ipykernel_5852\2680621923.py", line 153, in trade
TypeError: object of type 'NoneType' has no len()


In [2]:
import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time
import datetime



def init():
    mt5.initialize(path = r"C:\Program Files\Pepperstone MetaTrader 5\terminal64.exe")
    mt5.login(61220065 , password= 'Mahdi1400@' , server= 'Pepperstone-Demo')

# return balance , equity , profit
def info():

    account_info=mt5.account_info()

    balance = account_info.balance
    equity  = account_info.equity
    profit  = account_info.profit
    
    return balance , equity , profit
    

def symbol_info(symbol):
    dic_symbol_info = {}
    group_symbols=mt5.symbols_get(group='*USD*')
    for s in group_symbols:
        if s.name == symbol : 
            dic_symbol_info[s.name] =  s.ask , s.bid
            
    return dic_symbol_info  # برمیگرداند یک دیکشنری که کلدی ها نماد و در ولیو ها اسک و بید به صورت تاپل 


def buy(symbol , lot , tp_ , sl_ ):
    
    
    tp = abs(mt5.symbol_info_tick(symbol).bid + tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).ask - sl_ ) 
    pos = mt5.ORDER_TYPE_BUY







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC ,
    }
    print(request)   
    # send a trading request
    result = mt5.order_send(request)
    
    # check the execution result
    print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def sell(symbol , lot , tp_ , sl_ ):
    
    tp = abs(mt5.symbol_info_tick(symbol).ask - tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).bid + sl_ ) 
    pos = mt5.ORDER_TYPE_SELL







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC ,
    }
        
    # send a trading request
    result = mt5.order_send(request)
    # check the execution result
    # print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def trade(symbol , lot , tp , sl):
    numtrade = 0
    # up_numtrade = 0
    sl = sl 
    while True:
        
        positions_total = len(mt5.positions_get(symbol=symbol))
        positions_info  = mt5.positions_get(symbol=symbol)
        
        # print(positions_total)
        if  positions_total == 0 :
            
            th.Thread(target= buy , args= ( symbol, lot ,  tp ,  sl )).start()
            th.Thread(target= sell , args= ( symbol, lot ,  tp ,  sl)).start()
            
            numtrade = 2
            continue
        # print(positions_info[0].type)
        if  positions_total != None :
            if positions_total < numtrade :

                buy( symbol, lot ,  tp ,  sl )               
                sell( symbol, lot ,  tp ,  sl)
                                
                numtrade =+ 2
                continue
            

            
            numtrade = positions_total
        else:
            print("Positions not found")

        time.sleep(0.3)
        
        
def main():
    # دریافت زمان فعلی



    print(info())
    # th.Thread(target= trade   , args= ( 'USDJPY_o' ,  0.1 , 0.009   , 2  )).start()
    th.Thread(target= trade , args= ( 'GBPUSD' ,  0.1 , 0.00015 , 0.001)).start()
    # th.Thread(target= trade , args= ( 'NZDUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCAD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCHF_o' ,  0.1 , 0.00008 , 0.03 )).start()
    # th.Thread(target= trade , args= ( 'EURUSD_o' ,  0.1 , 0.00007 , 0.03 )).start()
    # th.Thread(target= trade , args= ( 'XAUUSD_o' ,  0.1 , 0.35 , 100 )).start()

    # trade(symbol='USDJPY_o' , lot= 0.1 ,tp= 0.009 ,sl= 0.006)
    # trade(symbol='GBPUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='NZDUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCAD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCHF_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='EURUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)


def close():
    
    while True:
        
        positions = mt5.positions_get()
        def close_position(position):

            tick = mt5.symbol_info_tick(position.symbol)

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "position": position.ticket,
                "symbol": position.symbol,
                "volume": position.volume,
                "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
                "price": tick.ask if position.type == 1 else tick.bid,  
                "deviation": 100,
                "magic": 234002,
                "comment": "Gu",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK ,
            }

            result = mt5.order_send(request)
            return result

        for position in positions:
            # print(position)
            # 
            close_position(position)


def run():


    while True:
        
          # توقف اجرا برای ۶۰ ثانیه
        current_time = datetime.datetime.now().time()
        print(current_time)
        # مقایسه زمان فعلی با زمان مورد نظر (ساعت 6:00 صبح)
        target_time = datetime.time(6, 1, 0)  # زمان مورد نظر
        target_close = datetime.time(19, 26, 0)  # زمان مورد نظر
        run_main = 0
        close_main = 0

        if current_time.hour == target_time.hour and run_main == 0  :
            run_main = 1

            main()
            
        
        if current_time.hour == target_close.hour and close_main == 0 :
            close_main = 1

            close()
            
            
        time.sleep(120)

init()
# main()
# close()
th.Thread(target= main).start()
# th.Thread(target= close).start()

(49999.4, 49995.6, -3.8)


{'action': 1, 'symbol': 'GBPUSD', 'volume': 0.1, 'type': 0, 'price': 1.25227, 'sl': 1.25127, 'tp': 1.2523900000000001, 'deviation': 100, 'magic': 234002, 'comment': 'Gu', 'type_time': 0, 'type_filling': 1}
1. order_send(): by GBPUSD 0.1 lots at 1.25227 with deviation=100 points
{'action': 1, 'symbol': 'GBPUSD', 'volume': 0.1, 'type': 0, 'price': 1.25225, 'sl': 1.2512500000000002, 'tp': 1.2523600000000001, 'deviation': 100, 'magic': 234002, 'comment': 'Gu', 'type_time': 0, 'type_filling': 1}{'action': 1, 'symbol': 'GBPUSD', 'volume': 0.1, 'type': 0, 'price': 1.25225, 'sl': 1.2512500000000002, 'tp': 1.2523600000000001, 'deviation': 100, 'magic': 234002, 'comment': 'Gu', 'type_time': 0, 'type_filling': 1}

1. order_send(): by GBPUSD 0.1 lots at 1.25225 with deviation=100 points
1. order_send(): by GBPUSD 0.1 lots at 1.25225 with deviation=100 points
{'action': 1, 'symbol': 'GBPUSD', 'volume': 0.1, 'type': 0, 'price': 1.25207, 'sl': 1.2510700000000001, 'tp': 1.2521900000000001, 'deviation'

In [1]:
import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time
import datetime



def init():
    mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 Terminal\terminal64.exe")
    mt5.login(89051664 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Demo')

# return balance , equity , profit
def info():

    account_info=mt5.account_info()

    balance = account_info.balance
    equity  = account_info.equity
    profit  = account_info.profit
    
    return balance , equity , profit
    

def symbol_info(symbol):
    dic_symbol_info = {}
    group_symbols=mt5.symbols_get(group='*USD*')
    for s in group_symbols:
        if s.name == symbol : 
            dic_symbol_info[s.name] =  s.ask , s.bid
            
    return dic_symbol_info  # برمیگرداند یک دیکشنری که کلدی ها نماد و در ولیو ها اسک و بید به صورت تاپل 


def buy(symbol , lot , tp_ , sl_ ):
    
    
    tp = abs(mt5.symbol_info_tick(symbol).bid + tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).ask - sl_ ) 
    pos = mt5.ORDER_TYPE_BUY







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
    print(request)   
    # send a trading request
    result = mt5.order_send(request)
    
    # check the execution result
    print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def sell(symbol , lot , tp_ , sl_ ):
    
    tp = abs(mt5.symbol_info_tick(symbol).ask - tp_ ) 
    sl = abs(mt5.symbol_info_tick(symbol).bid + sl_ ) 
    pos = mt5.ORDER_TYPE_SELL







    # lot = 1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "sl": sl ,
        "tp": tp ,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
        
    # send a trading request
    result = mt5.order_send(request)
    # check the execution result
    # print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
    # if result:
    #     if result.retcode != mt5.TRADE_RETCODE_DONE:
    #         print("2. order_send failed, retcode={}".format(result.retcode))
    #         # request the result as a dictionary and display it element by element
    #         result_dict=result._asdict()
    #         for field in result_dict.keys():
    #             print("   {}={}".format(field,result_dict[field]))
    #             # if this is a trading request structure, display it element by element as well
    #             if field=="request":
    #                 traderequest_dict=result_dict[field]._asdict()
    #                 for tradereq_filed in traderequest_dict:
    #                     print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))


def trade(symbol , lot , tp , sl):
    numtrade = 0
    # up_numtrade = 0
    sl = sl 
    while True:
        
        positions_total = len(mt5.positions_get(symbol=symbol))
        positions_info  = mt5.positions_get(symbol=symbol)
        
        # print(positions_total)
        if  positions_total == 0 :
            
            th.Thread(target= buy , args= ( symbol, lot ,  tp ,  sl )).start()
            th.Thread(target= sell , args= ( symbol, lot ,  tp ,  sl)).start()
            
            numtrade = 2
            continue
        # print(positions_info[0].type)
        if  positions_total != None :


            buy( symbol, lot ,  tp ,  sl )               
            sell( symbol, lot ,  tp ,  sl)
                            
            numtrade =+ 2
            time.sleep(0.5)
            continue
            
        
            
            numtrade = positions_total
        

        time.sleep(0.35)
        
        
def main():
    # دریافت زمان فعلی



    print(info())
    # th.Thread(target= trade   , args= ( 'USDJPY_o' ,  0.1 , 0.01   , 2   )).start()
    th.Thread(target= trade , args= ( 'GBPUSD_o' ,  0.1 , 0.00007 , 0.005)).start()
    # th.Thread(target= trade , args= ( 'NZDUSD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCAD_o' ,  0.1 , 0.00009 , 0.00006 )).start()
    # th.Thread(target= trade , args= ( 'USDCHF_o' ,  0.1 , 0.00007 , 0.03 )).start()
    # th.Thread(target= trade , args= ( 'EURUSD_o' ,  0.1 , 0.00007 , 0.03 )).start()


    # trade(symbol='USDJPY_o' , lot= 0.1 ,tp= 0.009 ,sl= 0.006)
    # trade(symbol='GBPUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='NZDUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCAD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='USDCHF_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)
    # trade(symbol='EURUSD_o' , lot= 0.1 ,tp= 0.00009 ,sl= 0.0006)


def close():
    

        
    positions = mt5.positions_get()
    def close_position(position):

        tick = mt5.symbol_info_tick(position.symbol)

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
            "price": tick.ask if position.type == 1 else tick.bid,  
            "deviation": 100,
            "magic": 234002,
            "comment": "Gu",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK ,
        }

        result = mt5.order_send(request)
        return result

    for position in positions:
        # print(position)
        # 
        close_position(position)


def run():


    while True:
        
          # توقف اجرا برای ۶۰ ثانیه
        current_time = datetime.datetime.now().time()
        print(current_time)
        # مقایسه زمان فعلی با زمان مورد نظر (ساعت 6:00 صبح)
        target_time = datetime.time(6, 1, 0)  # زمان مورد نظر
        target_close = datetime.time(19, 26, 0)  # زمان مورد نظر
        run_main = 0
        close_main = 0

        if current_time.hour == target_time.hour and run_main == 0  :
            run_main = 1

            main()
            
        
        if current_time.hour == target_close.hour and close_main == 0 :
            close_main = 1

            close()
            
            
        time.sleep(60)




init()
# main()
# close()
# th.Thread(target= main).start()
th.Thread(target= close).start()

C:\Users\delta2794900\AppData\Local\Temp\ipykernel_22672\853809246.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
df = pd.read_csv('1.csv' , delimiter=';')


In [ ]:
df